In [ ]:
setwd("C:/IE582")
train = read.csv("IE582_Fall20_ProjectTrain.csv",sep = ",", dec = ".")
test = read.csv("IE582_Fall20_ProjectTest.csv",sep = ",", dec = ".")
train$y = as.factor(train$y)

In [ ]:
library(randomForest)
library(mlbench)
library(caret)
library(ROSE)

In [ ]:
library(DMwR)
seed <- 7
set.seed(seed)
smoted_data <- SMOTE(y~., train, perc.over=100,perc.under = 200,k=1)

In [ ]:
table(smoted_data$y)

In [ ]:
x <- smoted_data[,1:60]
y <- smoted_data[,61]
metric <- "Accuracy"
mtry <- sqrt(ncol(x))
# Algorithm Tune (tuneRF)
set.seed(42)
bestmtry <- tuneRF(x, y, stepFactor=1.5, improve=1e-5, ntree=1000)
print(bestmtry)

In [ ]:
set.seed(1192)
control <- trainControl(method='repeatedcv',number=10,repeats=5,classProbs=TRUE,summaryFunction = twoClassSummary)
x <- smoted_data[,1:60]
y <- smoted_data[,61]
tunegrid <- expand.grid(.mtry=15)
rf_default <- train(y~.,data=smoted_data,method='rf',metric="ROC",tuneGrid=tunegrid,trControl=control)
print(rf_default)

predictions<-predict(rf_default,test, type="prob")

In [ ]:
# install the required packages first
require(jsonlite)
require(httr)
require(data.table)

get_token <- function(username, password, url_site){
    
    post_body = list(username=username,password=password)
    post_url_string = paste0(url_site,'/token/')
    result = POST(post_url_string, body = post_body)

    # error handling (wrong credentials)
    if(result$status_code==400){
        print('Check your credentials')
        return(0)
    }
    else if (result$status_code==201){
        output = content(result)
        token = output$key
    }

    return(token)
}



send_submission <- function(predictions, token, url_site, submit_now=F){
    
    format_check=check_format(predictions)
    if(!format_check){
        return(FALSE)
    }
    
    post_string="list("
    for(i in 1:length(predictions)){
        if(i<length(predictions)){
            post_string=sprintf("%s%s,",post_string,predictions[i])
        } else {
            post_string=sprintf("%s%s)",post_string,predictions[i])
        }
    }
    
    submission = eval(parse(text=post_string))
    json_body = jsonlite::toJSON(submission, auto_unbox = TRUE)
    submission=list(submission=json_body)
    print(submission)

    if(!submit_now){
        print("You did not submit.")
        return(FALSE)      
    }
    

    header = add_headers(c(Authorization=paste('Token',token,sep=' ')))
    post_url_string = paste0(url_site,'/submission/')
    result = POST(post_url_string, header, body=submission)
    
    if (result$status_code==201){
        print("Successfully submitted. Below you can see the details of your submission")
    } else {
        print("Could not submit. Please check the error message below, contact the assistant if needed.")
    }
    
    print(content(result))
    
}

check_format <- function(predictions){
    
    if(all(is.numeric(predictions)) & all(predictions<=1)){
        print("Format OK")
        return(TRUE)
    } else {
        print("Wrong format")
        return(FALSE)
    }
    
}

In [ ]:
# this part is main code
subm_url = 'http://46.101.121.83'

u_name = "Datatata"
p_word = "hT7HEBh1hxztje2f"
submit_now = TRUE

username = u_name
password = p_word

token = get_token(username=u_name, password=p_word, url=subm_url)
# this part is where you need to provide your prediction method/function or set of R codes
p=predictions[,2]
send_submission(p, token, url=subm_url, submit_now= submit_now)